# 其他工具：Grab、PyQuery


* 利用 Grab 套件的存取 HTML 資源
* 利用 PyQuery 套件的解析 HTML 格式


## 作業目標

將之前用 requests + beatifulsoup 實作的方式，改寫成 grab + pyquery，並且比較有哪些地方不同。





## requests + BeautifulSoup


In [8]:
import requests
import os
from bs4 import BeautifulSoup
from PIL import Image

url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
resp = requests.get(url, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)

# 決定要儲存的資料夾
output_dir = 'downloads'

# 假如資料夾不存在就新增一個資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 定位所有圖片的 tag
image_tags = soup.find(id='main-content').findChildren('a', recursive=False)
for img_tag in image_tags:
    # 取得所有圖片在第三方服務的 id
    if 'imgur' not in img_tag['href']:
        continue
    img_id = img_tag['href'].split('/')[-1]
    # 組合圖片而非網站的網址
    img_url = 'https://i.imgur.com/{}.jpg'.format(img_id)
    # 對圖片送出請求
    with requests.get(img_url, stream=True) as r:
        r.raise_for_status()
        # 檢查圖片副檔名
        img = Image.open(r.raw)
        img_savename = '{outdir}/{img_id}.{img_ext}'.format(
            outdir=output_dir, img_id=img_id, img_ext=img.format.lower())
        img.save(img_savename)
        print('Save image {}'.format(img_savename))

Save image downloads/Cgb5oo1.jpeg
Save image downloads/MgjHY4f.jpeg
Save image downloads/yeoRtAs.jpeg
Save image downloads/wsauNEW.jpeg
Save image downloads/IjiETcs.jpeg


## Grab + PyQuery

In [15]:
from grab import Grab
from pyquery import PyQuery as pq

url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
g = Grab()
g.cookies.set(name='over18',value='1',domain='ptt.cc')
resp1 = g.go(url)
doc = pq(resp1.body)
title = doc('title')
print(type(title), title.text())

<class 'pyquery.pyquery.PyQuery'> [正妹] 新垣結衣幾張 發錢 - 看板 Beauty - 批踢踢實業坊


In [58]:
div = doc('div')
a = div('a')
for i in a.items():
    if 'imgur' not in str(i.html()):
        continue
    img_id = i.html().split('/')[-1]
    print(img_id)

Cgb5oo1
MgjHY4f
yeoRtAs
wsauNEW
IjiETcs
Cgb5oo1
MgjHY4f
yeoRtAs
wsauNEW
IjiETcs
